In [2]:
import pandas as pd
import numpy as np
import pickle
from shapely.geometry import Point
import geopandas as gpd
import addfips
import pickle
af=addfips.AddFIPS()

<h1> Block Group Population Center </h1>

In [3]:
# Read in census tract population and coordinate data
pop_centers = pd.read_csv('../../Obstetric accessibility visualization/Indiana/BG_Pop_Center.csv')
# pop_centers = pd.read_csv('../../example1.csv')

In [4]:
pop_centers.head() 

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE
0,18,1,30100,1,996,40.873707,-85.032811
1,18,1,30100,2,1107,40.886144,-84.960755
2,18,1,30100,3,995,40.874783,-84.848103
3,18,1,30100,4,1491,40.848539,-84.905501
4,18,1,30200,1,1610,40.845430,-84.947257


In [5]:
# Georgia shapefile
indiana_BG = gpd.read_file('../../Obstetric accessibility visualization/Indiana/tl_2017_13_bg.shp')

In [6]:
pop_centers['COUNTYFP'] = pop_centers.apply(lambda row: str(int(row['COUNTYFP'])).zfill(3),axis=1)
pop_centers['FIPS'] = pop_centers.apply(lambda row: str(row['STATEFP']) + str(row['COUNTYFP']),axis=1)
pop_centers['Coordinates'] = pop_centers.apply(lambda row: (row['LONGITUDE'],row['LATITUDE']),axis=1)
pop_centers = pop_centers.set_index('FIPS')

In [7]:
pop_centers.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,Coordinates
FIPS,,,,,,,,
18001,18,001,30100,1,996,40.873707,-85.032811,"(-85.032811, 40.873707)"
18001,18,001,30100,2,1107,40.886144,-84.960755,"(-84.960755, 40.886144)"
18001,18,001,30100,3,995,40.874783,-84.848103,"(-84.848103, 40.874783)"
18001,18,001,30100,4,1491,40.848539,-84.905501,"(-84.905501, 40.848539)"
18001,18,001,30200,1,1610,40.845430,-84.947257,"(-84.947257, 40.84543)"


In [8]:
geometry = [Point(xy) for xy in zip(pop_centers['Coordinates'])]
pop_centers = gpd.GeoDataFrame(pop_centers, 
                        crs = indiana_BG.crs,
                        geometry = geometry)
pop_centers.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,Coordinates,geometry
FIPS,,,,,,,,,
18001,18,001,30100,1,996,40.873707,-85.032811,"(-85.032811, 40.873707)",POINT (-85.03281 40.87371)
18001,18,001,30100,2,1107,40.886144,-84.960755,"(-84.960755, 40.886144)",POINT (-84.96076 40.88614)
18001,18,001,30100,3,995,40.874783,-84.848103,"(-84.848103, 40.874783)",POINT (-84.84810 40.87478)
18001,18,001,30100,4,1491,40.848539,-84.905501,"(-84.905501, 40.848539)",POINT (-84.90550 40.84854)
18001,18,001,30200,1,1610,40.845430,-84.947257,"(-84.947257, 40.84543)",POINT (-84.94726 40.84543)


In [9]:
def geography_column(row):
    county = str(int(row['COUNTYFP'])).zfill(3)
    tract = str(int(row['TRACTCE'])).zfill(6)
    s = '1500000US18' + county + tract + str(int(row['BLKGRPCE']))
    return s
    
pop_centers['Geography'] = pop_centers.apply(lambda row: geography_column(row),axis=1)
pop_centers.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,Coordinates,geometry,Geography
FIPS,,,,,,,,,,
18001,18,001,30100,1,996,40.873707,-85.032811,"(-85.032811, 40.873707)",POINT (-85.03281 40.87371),1500000US180010301001
18001,18,001,30100,2,1107,40.886144,-84.960755,"(-84.960755, 40.886144)",POINT (-84.96076 40.88614),1500000US180010301002
18001,18,001,30100,3,995,40.874783,-84.848103,"(-84.848103, 40.874783)",POINT (-84.84810 40.87478),1500000US180010301003
18001,18,001,30100,4,1491,40.848539,-84.905501,"(-84.905501, 40.848539)",POINT (-84.90550 40.84854),1500000US180010301004
18001,18,001,30200,1,1610,40.845430,-84.947257,"(-84.947257, 40.84543)",POINT (-84.94726 40.84543),1500000US180010302001


In [11]:
pop_centers.to_csv('Cleaned_Block_Group_Pop_Centers.csv.csv', index=False)

<h1> Reproductive Age Population By Block Group</h1>

In [9]:
bg_age = pd.read_csv('../../Obstetric accessibility visualization/Indiana/ACSDT5Y2022.B01001-Data.csv')


In [10]:
bg_age.head()

,GEO_ID,NAME,B01001_001E,B01001_001M,B01001_002E,B01001_002M,B01001_003E,B01001_003M,B01001_004E,B01001_004M,...,B01001_045M,B01001_046E,B01001_046M,B01001_047E,B01001_047M,B01001_048E,B01001_048M,B01001_049E,B01001_049M,Unnamed: 100
0,Geography,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!Male:,Margin of Error!!Total:!!Male:,Estimate!!Total:!!Male:!!Under 5 years,Margin of Error!!Total:!!Male:!!Under 5 years,Estimate!!Total:!!Male:!!5 to 9 years,Margin of Error!!Total:!!Male:!!5 to 9 years,...,Margin of Error!!Total:!!Female:!!67 to 69 years,Estimate!!Total:!!Female:!!70 to 74 years,Margin of Error!!Total:!!Female:!!70 to 74 years,Estimate!!Total:!!Female:!!75 to 79 years,Margin of Error!!Total:!!Female:!!75 to 79 years,Estimate!!Total:!!Female:!!80 to 84 years,Margin of Error!!Total:!!Female:!!80 to 84 years,Estimate!!Total:!!Female:!!85 years and over,Margin of Error!!Total:!!Female:!!85 years and...,NaN
1,0400000US18,Indiana,6784403,*****,3366798,1272,209991,560,222313,2591,...,2058,154785,2334,108129,1809,71664,1894,86419,1967,NaN
2,1500000US180010301001,Block Group 1; Census Tract 301; Adams County;...,1288,301,656,183,37,51,44,31,...,13,21,22,38,42,0,13,58,58,NaN
3,1500000US180010301002,Block Group 2; Census Tract 301; Adams County;...,1268,292,634,164,41,43,58,63,...,74,36,26,33,31,20,33,114,119,NaN
4,1500000US180010301003,Block Group 3; Census Tract 301; Adams County;...,874,240,468,138,0,13,30,28,...,25,7,12,10,17,0,13,0,13,NaN


In [11]:
# Get reproductive age females
bg_age.rename(columns=bg_age.iloc[0],inplace=True)
#bg_age.drop(bg_age.index[0], inplace=True)
bg_repro_age = bg_age[['Geographic Area Name',
                            'Estimate!!Total:!!Female:!!18 and 19 years',
                            'Estimate!!Total:!!Female:!!20 years',
                            'Estimate!!Total:!!Female:!!21 years',
                            'Estimate!!Total:!!Female:!!22 to 24 years',
                            'Estimate!!Total:!!Female:!!25 to 29 years',
                            'Estimate!!Total:!!Female:!!30 to 34 years',
                            'Estimate!!Total:!!Female:!!35 to 39 years',
                            'Estimate!!Total:!!Female:!!40 to 44 years']]
bg_repro_age_copy=bg_repro_age.copy()
bg_repro_age_copy.drop(bg_repro_age.index[0], inplace=True)
bg_repro_age_copy = bg_repro_age_copy.set_index('Geographic Area Name')
bg_repro_age.head()

,Geographic Area Name,Estimate!!Total:!!Female:!!18 and 19 years,Estimate!!Total:!!Female:!!20 years,Estimate!!Total:!!Female:!!21 years,Estimate!!Total:!!Female:!!22 to 24 years,Estimate!!Total:!!Female:!!25 to 29 years,Estimate!!Total:!!Female:!!30 to 34 years,Estimate!!Total:!!Female:!!35 to 39 years,Estimate!!Total:!!Female:!!40 to 44 years
0,Geographic Area Name,Estimate!!Total:!!Female:!!18 and 19 years,Estimate!!Total:!!Female:!!20 years,Estimate!!Total:!!Female:!!21 years,Estimate!!Total:!!Female:!!22 to 24 years,Estimate!!Total:!!Female:!!25 to 29 years,Estimate!!Total:!!Female:!!30 to 34 years,Estimate!!Total:!!Female:!!35 to 39 years,Estimate!!Total:!!Female:!!40 to 44 years
1,Indiana,94476,49938,47697,135928,221409,215383,214601,206020
2,Block Group 1; Census Tract 301; Adams County;...,0,0,0,23,0,69,48,28
3,Block Group 2; Census Tract 301; Adams County;...,0,0,0,0,102,22,5,50
4,Block Group 3; Census Tract 301; Adams County;...,0,0,13,14,0,16,13,35


In [21]:
bg_repro_age_copy= bg_repro_age_copy.iloc[1:]
bg_repro_age_copy = bg_repro_age_copy.astype(int)
bg_repro_age_copy['Total Female, 18-44'] = bg_repro_age_copy.iloc[:,1:8].sum(axis=1)
bg_repro_age_copy=bg_repro_age_copy.reset_index()

In [22]:
bg_repro_age_copy.to_csv('../../Obstetric accessibility visualization/Indiana/RA_Count_BlockGroup.csv', index=False)

In [ ]:
# get population of reproductive age women in census tract
p = {}
total=0
for b in bg_repro_age_copy.index:
    p[b] = int(bg_repro_age_copy['Total Female, 18-44'][b])
    total+=p[b]
total

In [ ]:
# save dictionary
with open('../../Obstetric accessibility visualization/Indiana/p_BG.pkl', 'wb') as fp:
    pickle.dump(p, fp)

In [ ]:
import pickle

with open('../../Obstetric accessibility visualization/Indiana/p_BG.pkl', 'rb') as file:
    df = pickle.load(file)


In [ ]:
df

<h1>Reproductive Age Population By Census Tract </h1>

In [ ]:
bg_age = pd.read_csv('../../Obstetric accessibility visualization/Indiana/RA_CensusTractLevel.csv')
# https://data.census.gov/table/ACSDT5Y2022.B01001?t=Age%20and%20Sex&g=040XX00US18$1400000

In [ ]:
bg_age.head()


In [ ]:
# Get reproductive age females
bg_age.rename(columns=bg_age.iloc[0],inplace=True)
#bg_age.drop(bg_age.index[0], inplace=True)
bg_repro_age = bg_age[['Geographic Area Name',
                            'Estimate!!Total:!!Female:!!18 and 19 years',
                            'Estimate!!Total:!!Female:!!20 years',
                            'Estimate!!Total:!!Female:!!21 years',
                            'Estimate!!Total:!!Female:!!22 to 24 years',
                            'Estimate!!Total:!!Female:!!25 to 29 years',
                            'Estimate!!Total:!!Female:!!30 to 34 years',
                            'Estimate!!Total:!!Female:!!35 to 39 years',
                            'Estimate!!Total:!!Female:!!40 to 44 years']]
bg_repro_age_copy=bg_repro_age.copy()
bg_repro_age_copy.drop(bg_repro_age.index[0], inplace=True)
bg_repro_age_copy = bg_repro_age_copy.set_index('Geographic Area Name')
bg_repro_age.head()

In [ ]:
bg_repro_age_copy= bg_repro_age_copy.iloc[1:]
bg_repro_age_copy = bg_repro_age_copy.astype(int)
bg_repro_age_copy['Total Female, 18-44'] = bg_repro_age_copy.iloc[:,1:8].sum(axis=1)

In [ ]:
bg_repro_age_copy.head()

<h1> Reproductive Age Population By County </h1>

In [ ]:
bg_age = pd.read_csv('../../Obstetric accessibility visualization/Indiana/RA_CountyLevel.csv')
#https://data.census.gov/table/ACSDT5Y2022.B01001?t=Age%20and%20Sex&g=040XX00US18$0500000

In [ ]:
bg_age.head()


In [ ]:
# Get reproductive age females
bg_age.rename(columns=bg_age.iloc[0],inplace=True)
#bg_age.drop(bg_age.index[0], inplace=True)
bg_repro_age = bg_age[['Geographic Area Name',
                            'Estimate!!Total:!!Female:!!18 and 19 years',
                            'Estimate!!Total:!!Female:!!20 years',
                            'Estimate!!Total:!!Female:!!21 years',
                            'Estimate!!Total:!!Female:!!22 to 24 years',
                            'Estimate!!Total:!!Female:!!25 to 29 years',
                            'Estimate!!Total:!!Female:!!30 to 34 years',
                            'Estimate!!Total:!!Female:!!35 to 39 years',
                            'Estimate!!Total:!!Female:!!40 to 44 years']]
bg_repro_age_copy=bg_repro_age.copy()
bg_repro_age_copy.drop(bg_repro_age.index[0], inplace=True)
bg_repro_age_copy = bg_repro_age_copy.set_index('Geographic Area Name')
bg_repro_age.head()

In [ ]:
bg_repro_age_copy= bg_repro_age_copy.iloc[1:]
bg_repro_age_copy = bg_repro_age_copy.astype(int)
bg_repro_age_copy['Total Female, 18-44'] = bg_repro_age_copy.iloc[:,1:8].sum(axis=1)
bg_repro_age_copy.head()